In [2]:
import os
from datetime import datetime
import numpy as np
import h5py
import re

In [3]:
# where are located old precomputed ratio files
dir = r'D:\radary_cartesian'
# output directory (should be the same as outdir in config file)
outdir = r'C:\Users\marti\kinit\radar-dataset-preprocess\output'
# where are located raw hdf files
rootdir = r'C:\Users\marti\kinit\radar-dataset-preprocess\data\2016-01-11'

In [4]:
# get paths to all hdf files in dir and its subdirectories
filepaths = []
for dirname, dirs, files in os.walk(dir):
    for filename in files:
        filename_without_extension, extension = os.path.splitext(filename)
        if extension == '.h5':
            filepaths.append(os.path.join(dirname, filename))

filepaths_root = []
for dirname, dirs, files in os.walk(rootdir):
    for filename in files:
        filename_without_extension, extension = os.path.splitext(filename)
        if extension == '.hdf':
            filepaths_root.append(os.path.join(dirname, filename))

In [5]:
print(filepaths_root[4])

C:\Users\marti\kinit\radar-dataset-preprocess\data\2016-01-11\T_PAGZ41_C_LZIB_20160111162000.hdf


In [6]:
print(filepaths[4])

D:\radary_cartesian\20160105.h5


In [7]:
# load ratios file from dir
ratios = np.array([], dtype=np.float64)
timestamps = np.array([], dtype=np.datetime64)
for file in filepaths:
    hf = h5py.File(file, 'r')
    # save them to variables
    ratios = np.append(ratios, hf['ratios'])
    timestamps = np.append(timestamps, [np.datetime64(datetime.strptime(item.decode(), '%Y-%m-%dT%H:%M:%S.%fZ')) for item in hf['timestamps']])
    hf.close()
    print(f'Done file number {filepaths.index(file)}/{len(filepaths)}.')

Done file number 0/1245.
Done file number 1/1245.
Done file number 2/1245.
Done file number 3/1245.
Done file number 4/1245.
Done file number 5/1245.
Done file number 6/1245.
Done file number 7/1245.


Done file number 8/1245.
Done file number 9/1245.
Done file number 10/1245.
Done file number 11/1245.
Done file number 12/1245.
Done file number 13/1245.
Done file number 14/1245.
Done file number 15/1245.
Done file number 16/1245.
Done file number 17/1245.
Done file number 18/1245.
Done file number 19/1245.
Done file number 20/1245.
Done file number 21/1245.
Done file number 22/1245.
Done file number 23/1245.
Done file number 24/1245.
Done file number 25/1245.
Done file number 26/1245.
Done file number 27/1245.
Done file number 28/1245.
Done file number 29/1245.
Done file number 30/1245.
Done file number 31/1245.
Done file number 32/1245.
Done file number 33/1245.
Done file number 34/1245.
Done file number 35/1245.
Done file number 36/1245.
Done file number 37/1245.
Done file number 38/1245.
Done file number 39/1245.
Done file number 40/1245.
Done file number 41/1245.
Done file number 42/1245.
Done file number 43/1245.
Done file number 44/1245.
Done file number 45/1245.
Done file numb

In [8]:
ratios.shape

(355761,)

In [9]:
timestamps.shape

(355761,)

In [10]:
ratios[2]

0.0025

In [11]:
timestamps[2058]

numpy.datetime64('2016-01-10T04:35:00.000000')

In [12]:
timestamps = np.array([np.datetime_as_string(n) for n in timestamps])
filepaths_new = []

# finds filepaths in rootdir for given timestamps
for timestamp in timestamps:
    idx = int(np.where(timestamps==timestamp)[0])
    timestamp = re.split('[-T:.]', timestamp)
    timestamp = ''.join(timestamp[:-1])
    if any(timestamp in s for s in filepaths_root):
        filepaths_new.append([s for s in filepaths_root if timestamp in s][0])
        print(f'Appended {timestamp}')
    else:
        filepaths_new.append('')
        
    if idx % 1000 == 0:
        print(f'{idx} out of {len(timestamps)} done.')

0 out of 355761 done.
1000 out of 355761 done.
2000 out of 355761 done.
Appended 20160111160000
Appended 20160111160500
Appended 20160111161000
Appended 20160111161500
Appended 20160111162000
Appended 20160111162500
Appended 20160111163000
Appended 20160111163500
Appended 20160111164000
Appended 20160111164500
Appended 20160111165000
Appended 20160111165500
Appended 20160111170000
Appended 20160111170500
Appended 20160111171000
Appended 20160111171500
Appended 20160111172000
Appended 20160111172500
Appended 20160111173000
3000 out of 355761 done.


KeyboardInterrupt: 

In [13]:
is_empty = 0
for file in filepaths_new:
    if file == '':
        print('Filepath empty.')
        is_empty += 1

if is_empty > 0:
    print(f'Total {is_empty} out of {len(filepaths_new)} filepaths are empty. Proceed with caution.')
else:
    print('All filepaths found successfully.')

Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath empty.
Filepath

In [14]:
# this chunk is necessary to proceed only if we didnt find all filepaths in rootdir
filepaths_new_mask = np.zeros(len(ratios), dtype=np.int64)
for file in filepaths_new:
    if file == '':
        filepaths_new_mask[filepaths_new.index(file)] = 0
    else:
        filepaths_new_mask[filepaths_new.index(file)] = 1

correct_idx = filepaths_new_mask.nonzero()[0]
ratios = ratios[correct_idx]
timestamps = timestamps[correct_idx]
filepaths_new = [filepaths_new[i] for i in correct_idx]

In [15]:
hf = h5py.File(os.path.join(outdir, 'ratios.h5'), 'a')
hf.create_dataset('ratios', data=ratios, chunks=True)
timestamps_utf8 = np.array([(n+'Z').encode('utf-8') for n in timestamps])
hf.create_dataset('timestamps', data=timestamps_utf8, chunks=True)
hf.create_dataset('filepaths', data=filepaths_new, chunks=True)
hf.close()